In [1]:
## import python modules
import numpy as np
import xarray as xr
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import glob
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.util import add_cyclic_point
import matplotlib
import matplotlib.cm as cm
from matplotlib.colors import BoundaryNorm
import datetime as dt


from matplotlib.animation import FuncAnimation

import matplotlib

from PIL import Image, ImageDraw
from matplotlib import animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
import matplotlib.image as mgimg
from collections import Counter
from global_land_mask import globe
import scipy.optimize as opt

from collections import Counter
from itertools import compress
from collections import OrderedDict

In [ ]:
# set paths to read buoyancy and precipitation data

In [2]:
buoy_path ='/neelin2020/ERA-5_buoy/layer_thetae/era5_2layers_thetae_' 
precip_path='/neelin2020/TRMM_3B42/TRMM.3hrly.3B42.'
number=[]
offset = []

In [243]:
number = []
offset = []
non_landfall = []
landfall = []
tropics = []
class hurricance:
    
    def __init__(self,year,paths):
        self.year=year
        self.buoy_path=paths[0]
        self.precip_path=paths[1]
        #self.buoy_path=paths[2]
        self.offset=10 ## box size in deg. lat/lon around feature center
        offset.append(self.offset)
    def __extract_dates(self,date_array):
        dates=[]
        for n in range(date_array.shape[0]):
            yr,mo,dy,hr=np.int_(date_array[n])
            dates.append(dt.datetime(yr,mo,dy,hr))
        
        return dates    
    
    def extract_tracks(self):
        
        ### read Tempest Extremes Tracks for ERA-5
        fili='/neelin2020/TempestExtremes/TC/ERA5_TC_tracks_20200824.txt'
        
        f=open(fili,'r')
        lines = f.readlines()    # Read file and close
        f.close()

        i=0
        self.tc_tracks={}
        ctr=0

        while (i < (len(lines)-1) ):
            line = lines[i]
            i = i+1
            linesplit = line.strip().split('\t')
            if linesplit[0] == 'start' :
                tracklength = int(linesplit[1])
                trackarray = np.genfromtxt(lines[i:(i+tracklength)])
                dates=self.__extract_dates(np.int_(trackarray[:,-4:]))
                lon=trackarray[:,2]
                lat=trackarray[:,3]
                psl=trackarray[:,4]*1e-2 ### in hPa
                max_wind=trackarray[:,5]
                
        #         if(np.logical_and(dates[0].year>start_year, dates[-1].year<=end_year)): ## for start and end years
                if(dates[0].year==self.year & dates[-1].year==self.year):
                    self.tc_tracks[ctr]={}
                    self.tc_tracks[ctr]['lat']=lat
                    self.tc_tracks[ctr]['lon']=lon
                    self.tc_tracks[ctr]['dates']=dates
                    self.tc_tracks[ctr]['psl']=psl
                    self.tc_tracks[ctr]['maxwind']=max_wind
                    ctr+=1
        print('There are {:d} Tropical Cyclones in {:d}'.format(ctr,self.year))
        number.append(ctr)
    def read_data(self):
        
        '''
        This is the function that reads the buoyancy 
        fields from the ERA5 product.
        '''

        self.ds_comp={}
        
        for key in self.tc_tracks.keys():

            fil_era5=glob.glob(self.buoy_path+'2002_01_01.nc')[0]
            ds_buoy=xr.open_dataset(fil_era5)
            ds_buoy.close()
            
            dates=self.tc_tracks[key]['dates']
            
            nan_array=np.zeros((len(dates),ds_buoy.lat.size,ds_buoy.lon.size))
            nan_array[:]=np.nan    
                

            self.ds_comp[key] = xr.Dataset(
                data_vars=dict(
                    #crh=(["time","lat", "lon"], nan_array.copy()),
                    CAPE =(["time","lat", "lon"], nan_array.copy()),
                    Subsaturation=(["time","lat", "lon"], nan_array.copy()),
                    Buoyancy=(["time","lat", "lon"], nan_array.copy()),
                    EQBL=(["time","lat", "lon"], nan_array.copy()),
                    EQSLFT=(["time","lat", "lon"], nan_array.copy()),
                    EQULFT=(["time","lat", "lon"], nan_array.copy()),
                    precipitation=(["time","lat", "lon"], nan_array.copy()),
                ),
                coords=dict(
                    lon=(ds_buoy.lon),
                    lat=(ds_buoy.lat),
                    time=dates,
                ))
                
    #print(self.buoy_path+'2002_01_01.nc')     
                                    
            for i,j,k in zip(self.tc_tracks[key]['lat'],self.tc_tracks[key]['lon'],self.tc_tracks[key]['dates']):
                try:
                    fil_era5=glob.glob(self.buoy_path+dt.datetime.strftime(k,'%Y_%m_%d')+'.nc')[0]
                    fil_precip=glob.glob(self.precip_path+dt.datetime.strftime(k,'%Y%m*')+'.nc')[0]
                except:
                    print('Files not found')
                    break
                

                ### extract lat, lon ranges for each date ###
                ds_buoy=xr.open_dataset(fil_era5)
                ds_precip=xr.open_dataset(fil_precip)
                ds_precip=ds_precip.rename({'latitude':'lat',
                                           'longitude':'lon'})
                
                la=ds_buoy.lat
                lc=ds_buoy.lon

                loc_dict=dict(lat=la[(la>=i-self.offset) & (la<=i+self.offset)],
                       lon=lc[(lc>=j-self.offset) & (lc<=j+self.offset)])

                loc_dict_time=dict(time=k,
                                   lat=la[(la>=i-self.offset) & (la<=i+self.offset)],
                       lon=lc[(lc>=j-self.offset) & (lc<=j+self.offset)])

                
                if len(loc_dict['lat'])==0:
                    print('Latitude out of bounds')
                    break

                thetae_bl=ds_buoy.thetae_bl.sel(time=k,method='nearest').loc[loc_dict]
                #print(thetae_bl)
                thetae_lt=ds_buoy.thetae_lt.sel(time=k,method='nearest').loc[loc_dict]
                thetae_sat_lt=ds_buoy.thetae_sat_lt.sel(time=k,method='nearest').loc[loc_dict]
                #print(thetae_sat_lt)
#                 print([loc_dict])
                CAPE_new = ((thetae_bl-thetae_sat_lt)/thetae_sat_lt)*340
                SUBSAT_new = ((thetae_sat_lt-thetae_lt)/thetae_sat_lt)*340
                BUOY_new = (9.8/(340*3))*((.5*CAPE_new)-(.5*SUBSAT_new))
                #crh_inst=cwv/cwv_sat
                self.ds_comp[key]['CAPE'].loc[loc_dict_time]=CAPE_new
                
                self.ds_comp[key]['Subsaturation'].loc[loc_dict_time]=SUBSAT_new
                
                self.ds_comp[key]['Buoyancy'].loc[loc_dict_time]=BUOY_new
                
                self.ds_comp[key]['EQBL'].loc[loc_dict_time]=thetae_bl
                self.ds_comp[key]['EQSLFT'].loc[loc_dict_time]=thetae_sat_lt
                self.ds_comp[key]['EQULFT'].loc[loc_dict_time]=thetae_lt
                prc_inst=ds_precip.precip_trmm.sel(time=k,method='nearest').loc[loc_dict]
                self.ds_comp[key]['precipitation'].loc[loc_dict_time]=prc_inst
                

                
                ds_buoy.close()
                ds_precip.close()
            
            self.ds_comp[key].close()
            print(key, 'Max wind: {:.2f} m/s, Min psl: {:.2f} hPa'.format(max(self.tc_tracks[key]['maxwind']), min(self.tc_tracks[key]['psl'])))
            print(tcs.ds_comp[key].time.dt.strftime("%Y%m%d-%H").isel(time=0).item())
            long_array=tcs.tc_tracks[key]['lon'] #taking latitude and longitude arrays
            lati_array=tcs.tc_tracks[key]['lat']
            breaker = 0
            breakers = 0
            for lan in range(len(long_array)):
                if long_array[lan] >= 180: #since the landmask array takes negatives instead of 360
                    long_array[lan] = long_array[lan]-360
                if globe.is_land(lati_array[lan], long_array[lan]) == True: #if one value is on the landmask
                    
                    if breaker == 0:
                        breaker = breaker+1 #so it doesn't print landfall for every point
                        print("Landfalls at some point")
                        landfall.append(key) #append to landfall list
                if globe.is_land(lati_array[lan], long_array[lan]) == False: #for storms that never hit the landmask
                    breakers = breakers + 1
                if long_array[lan] < 0:
                    long_array[lan] = long_array[lan]+360
                if breakers == len(lati_array): #if every position is not over land
                    non_landfall.append(key)#append to nonlandfall list
            passer = 0 
            for lin in range(len(lati_array)):
                if (lati_array[lin] >= (-35+offset[0])) and (lati_array[lin] <= (35-offset[0])):
                    passer = passer + 1
                    #if the box around a storm passes 35 degree latitude or not
                if passer == len(lati_array): #if all positions are within the tropics
                    print("Stays in the Tropics")
                    tropics.append(key)
            #so I know what basin a storm belongs to (isn't perfect)
            if long_array[0] >= 261 and long_array[0] <= 359:
                print("Atlantic")
            elif long_array[0] >= 220 and long_array[0] < 261 and lati_array[0] >= 0:
                print("East Pacific")
            elif long_array[0] >= 180 and long_array[0] < 220 and lati_array[0] >= 0:
                print("Central Pacific")
            elif long_array[0] >= 100 and long_array[0] < 180 and lati_array[0] >= 0:
                print("West Pacific")
            elif long_array[0] >= 45 and long_array[0] < 100 and lati_array[0] >= 0:
                print("North Indian")
            elif long_array[0] >= 20  and long_array[0] < 90 and lati_array[0] < 0:
                print("Southwest Indian")
            elif long_array[0] >= 90  and long_array[0] < 160 and lati_array[0] < 0:
                print("Australian")
            elif long_array[0] >= 160  and long_array[0] < 300 and lati_array[0] < 0:
                print("South Pacific")
            else:
                print("That's not right!")
            print("\n")

In [244]:
%%time
### read 
year=2013 ## enter year here. We have years 2002-2013 processed
tcs=hurricance(year,[buoy_path,precip_path]) #reads dict into file
tcs.extract_tracks() #extracts information
tcs.read_data()
adjust=2.5 #determines how big an outter box is wanted around environment

There are 77 Tropical Cyclones in 2013
0 Max wind: 21.37 m/s, Min psl: 994.40 hPa
20130103-00
Stays in the Tropics
West Pacific


1 Max wind: 28.77 m/s, Min psl: 960.86 hPa
20130106-06
Australian


2 Max wind: 18.00 m/s, Min psl: 999.58 hPa
20130111-00
Stays in the Tropics
Southwest Indian


Latitude out of bounds
3 Max wind: 28.17 m/s, Min psl: 935.96 hPa
20130115-18
Landfalls at some point
West Pacific


4 Max wind: 17.23 m/s, Min psl: 997.26 hPa
20130116-06
Landfalls at some point
Stays in the Tropics
Southwest Indian


5 Max wind: 22.71 m/s, Min psl: 986.93 hPa
20130118-18
South Pacific


6 Max wind: 31.73 m/s, Min psl: 963.42 hPa
20130124-18
Southwest Indian


7 Max wind: 15.93 m/s, Min psl: 1003.59 hPa
20130208-00
South Pacific


8 Max wind: 29.47 m/s, Min psl: 963.46 hPa
20130209-12
Southwest Indian


9 Max wind: 22.97 m/s, Min psl: 992.88 hPa
20130209-18
South Pacific


10 Max wind: 29.01 m/s, Min psl: 973.46 hPa
20130216-06
Landfalls at some point
Southwest Indian


11 Max win

In [245]:
def conversionudf(degree):
    '''
    Converts degrees latitude/longitude into kilometers
    '''
    box_conv = degree*111
    return box_conv
def backconversionudf(km):
    '''
    Converts kilometeres into degrees latitude/longitude
    '''
    box_conv = km/111
    return box_conv
def x_round(x):
    '''
    Rounds data to nearest .25 (as the box has a resolution of .25 degrees)
    '''
    return round(x*4)/4
def cleanudf(time,newtime,size,latitude):
    '''
    Takes data extracted from loop for TCSize, and adjusts the time and sizes to the ERA5 data here
    '''
    mask = []
    indicator = 0
    #if the length of the TCSize time is greater than the ERA5 data
    if len(time) > len(newtime):
        x=time
        y=newtime
    else:
        x=newtime
        y=time
        #find the ERA5 time that matches the new time if it is longer
        while newtime[0+indicator]!=time[0]: 
            indicator = indicator+1
    for i in range(len(x)):
        for j in range(len(y)):
            if x[i] == y[j] and x[i-1]!=x[i]:

                mask.append(i)
                #mask created of which times line up
            else:
                continue

    
    mask_sizer=[]
    #applies time mask to sizes
    for x in mask:
        mask_sizer.append(size[x-indicator])
    mask_latitude=[]
    for x in mask:
        mask_latitude.append(latitude[x-indicator])

    
    new_size = []
    #applies converts sizes into their degree latitude/longitude
    for i in range(len(mask_sizer)):
        new_size.append(backconversionudf(mask_sizer[i]))
    mask_2 = []
    #removes data that will not have a complete environmental box around it
    for i in range(len(mask_latitude)):
        if float(mask_latitude[i])+new_size[i] < (34.75-adjust):
            mask_2.append(i)
    mask2_sizer=[]
    #rounds the collected data to the nearest .25 degree
    for x in mask_2:
        mask2_sizer.append(x_round(backconversionudf(mask_sizer[x])))
    
    mask2_latitude=[]
    #appends this change to the latitude mask
    for x in mask_2:
        mask2_latitude.append(mask_latitude[x])
    mask2_time=[]
    #gets the time indices used
    for x in mask_2:

        mask2_time.append(newtime[x])

    time_indice = []
    #applies these time indices to the newtime array, and can be used for latitude, size, and time
    for x in mask2_time:
        for y in newtime:
            if y == x:

                time_indice.append(newtime.index(y))
    new_mask= [(x>.25) for x in mask2_sizer] #avoids any storm sizes that are .25 degrees or smaller (would only use 4 data points and loses accuracy)
    mask2_latitude=list(compress(mask2_latitude, new_mask)) #apply mask to latitude
    mask2_sizer=list(compress(mask2_sizer, new_mask)) #apply mask to size
    time_indice = list(compress(time_indice, new_mask)) #apply mask to time indice
    
    return mask2_time,mask2_sizer,mask2_latitude,time_indice



In [252]:
basins="NATL"
#Basin choices include
#North Atlantic (NATL)
#North Eastern Pacific (NEPAC)
#North Indian Ocean (NIO)
#South Indian Ocean (SIO)
#South Pacific Ocean (SPO)
#Western North Pacific (WNP)
with open('{}_TCSize.dat'.format(basins)) as f:
    mylist = list(f)
looker = "AL142013,            MELISSA,     26,\n" #have to manually find the storm and name, document including it is included
tc_number = 68 #choose TC associated with the name
area=mylist.index(looker) #takes the indices
chooser = int(looker[-5:-2]) #takes the number of indices at the end of the string
compare_time = []
latituder = []
sizer = []
for i in range(chooser):
    time=mylist[area+1+i][4:8]+"-"+mylist[area+1+i][11:13] #takes the time associated with each timestep
    compare_time.append(time)
    #sizer.append(float(mylist[area+1+i][-8:-1])) #for the entire storm's width
    sizer.append(float(mylist[area+1+i][-40:-34])) #for the TS force winds width
    latituder.append(mylist[area+1+i][17:21]) #takes the latitudes for each time step

time_array=[]
g = np.int_(np.linspace(0,tcs.ds_comp[tc_number].precipitation.time.size-1))
for t in range(max(g)+1):
    v = tcs.ds_comp[tc_number].time.dt.strftime("%m%d-%H").isel(time=t).item()
    time_array.append(v) #time array to use for newtime in the cleanudf
time_storm, size_storm, latitude_storm, indice_storm = cleanudf(compare_time,time_array,sizer,latituder)
lat_array=tcs.tc_tracks[tc_number]['lat'] #takes latitude 
lon_array=tcs.tc_tracks[tc_number]['lon'] #takes longitude
print(indice_storm)
print(size_storm)
factor="CAPE/Subsaturation"
#Choices Include:
#CAPE/Subsaturation
#EQBL/EQSLFT/EQULFT
#Buoyancy
#Precipitation

[0, 1, 2]
[1.25, 1.5, 1.5]


In [253]:
l=0 #since I need to call the sizes independent of the time array
for t in indice_storm: #takes each indice
    print(t)

    rounder = size_storm[l] #so it's easier to call
    print(rounder)
    g = np.int_(np.linspace(0,tcs.ds_comp[tc_number].precipitation.time.size-1)) #takes the integer of each timestep

    lat_array=tcs.tc_tracks[tc_number]['lat']
    lon_array=tcs.tc_tracks[tc_number]['lon']
    psl=tcs.tc_tracks[tc_number]['psl']
    maxwind=tcs.tc_tracks[tc_number]['maxwind']

    off_setter = rounder+adjust #the size environment around it

    interner = rounder #value used for inner comparing
    off_check = off_setter-interner #the value in between the two

    off_range = (2*off_setter)-.25 #the size of the outter box
    eqbl= tcs.ds_comp[tc_number].EQBL.isel(time=t) #taking the values for equivalent pot. temp. in the ABL
    eqblada=eqbl.stack(x=['lon','lat'])
    eqbla=eqblada[eqblada.notnull()] #so I can manipulate the values for this factor and take the inside and outside

    eqslft= tcs.ds_comp[tc_number].EQSLFT.isel(time=t) #taking the values for equivalent sat. pot. temp. in the LFT
    eqslftada=eqslft.stack(x=['lat','lon'])
    eqslfta=eqslftada[eqslftada.notnull()]

    equlft= tcs.ds_comp[tc_number].EQULFT.isel(time=t) #taking the values for equivalent unsat. pot. temp. in the LFT
    equlftada=equlft.stack(x=['lat','lon'])
    equlfta=equlftada[equlftada.notnull()]

    subs= tcs.ds_comp[tc_number].Subsaturation.isel(time=t) #taking the values for subsaturation in the LFT
    subsada_stacked=subs.stack(x=['lat','lon'])
    subsa=subsada_stacked[subsada_stacked.notnull()]

    cape= tcs.ds_comp[tc_number].CAPE.isel(time=t) #taking the values for CAPE in the LFT
    capada_stacked=cape.stack(x=['lat','lon'])
    capea=capada_stacked[capada_stacked.notnull()]

    buoy= tcs.ds_comp[tc_number].Buoyancy.isel(time=t) #taking the buoyancy for buoyancy in the LFT
    buoyada=buoy.stack(x=['lat','lon'])
    buoya=buoyada[buoyada.notnull()]

    preci= tcs.ds_comp[tc_number].precipitation.isel(time=t) #taking the values for precipitation
    preciada=preci.stack(x=['lat','lon'])
    precia=preciada[preciada.notnull()]

    if len(eqbla.lon.values) == 0: #skips values that have no below 34.75 degree latitude absolute values
        continue

    max_long=lat_array[t]+off_setter
    min_long=lat_array[t]-off_setter
    #For the outter ring
    if (max_long <= 34.75) or (min_long >=-34.75): #removes any absolute value that goes above 34.75 degrees

        r=lon_array[t]+interner #max longitude inside
        qq=r
        qq_check=r+off_check #max longitude outside
        
        o=lon_array[t]-interner #min longitude inside
        pp_check=o
        pp=o-off_check #min longitude outside
        
        rr=lat_array[t]+interner #max latitude inside
        q=rr
        q_check=rr+off_check #max latitude outside
        
        oo=lat_array[t]-interner #min latitude inside
        p_check=oo
        p=oo-off_check #min latitude outside
        



        ches=eqbla.where((eqbla.lon>=pp) & (eqbla.lon<=pp_check) | (eqbla.lon>=qq) & (eqbla.lon<=qq_check))
        eqbl_check=ches.where((eqbla.lat>=p) & (eqbla.lat<=p_check)| (eqbla.lat>=q) & (eqbla.lat<=q_check))
        chess=eqslfta.where((eqslfta.lon>=pp) & (eqslfta.lon<=pp_check) | (eqslfta.lon>=qq) & (eqslfta.lon<=qq_check))
        eqslft_check=chess.where((eqslfta.lat>=p) & (eqslfta.lat<=p_check)| (eqslfta.lat>=q) & (eqslfta.lat<=q_check))
        chesser=equlfta.where((equlfta.lon>=pp) & (equlfta.lon<=pp_check) | (equlfta.lon>=qq) & (equlfta.lon<=qq_check))
        equlft_check=chesser.where((equlfta.lat>=p) & (equlfta.lat<=p_check)| (equlfta.lat>=q) & (equlfta.lat<=q_check))
        chessest=subsa.where((subsa.lon>=pp) & (subsa.lon<=pp_check) | (subsa.lon>=qq) & (subsa.lon<=qq_check))
        subs_check=chessest.where((subsa.lat>=p) & (subsa.lat<=p_check)| (subsa.lat>=q) & (subsa.lat<=q_check))
        chessester=capea.where((capea.lon>=pp) & (capea.lon<=pp_check) | (capea.lon>=qq) & (capea.lon<=qq_check))
        cape_check=chessester.where((capea.lat>=p) & (capea.lat<=p_check)| (capea.lat>=q) & (capea.lat<=q_check))
        chessesterer=buoya.where((buoya.lon>=pp) & (buoya.lon<=pp_check) | (buoya.lon>=qq) & (buoya.lon<=qq_check))
        buoy_check=chessesterer.where((buoya.lat>=p) & (buoya.lat<=p_check)| (buoya.lat>=q) & (buoya.lat<=q_check))
        precip_chess=precia.where((precia.lon>=pp) & (precia.lon<=pp_check) | (precia.lon>=qq) & (precia.lon<=qq_check))
        preci_check=precip_chess.where((precia.lat>=p) & (precia.lat<=p_check)| (precia.lat>=q) & (precia.lat<=q_check))

    else:
        continue
    eqblext = np.nanmean(eqbl_check) #take the average of the outter ring
    eqslftext = np.nanmean(eqslft_check)
    equlftext = np.nanmean(equlft_check)
    subsext = np.nanmean(subs_check)
    capeext = np.nanmean(cape_check)
    buoyext = np.nanmean(buoy_check)
    preciext = np.nanmean(preci_check)

    max_long=lat_array[t]+off_setter
    min_long=lat_array[t]-off_setter
    #For the inner box of range interner
    if (max_long <= 34.75) or (min_long >=-34.75): #removes any absolute value that goes above 34.75 degrees
        r=lon_array[t]+interner #max longitude of inner box
        o=lon_array[t]-interner #min longitude of inner box
        pp=o
        qq=r
        rr=lat_array[t]+interner #max latitude of inner box
        oo=lat_array[t]-interner #max latitude of inner box
        p=oo
        q=rr

        eqbl_check = eqbla.where((eqbla.lon>=pp) & (eqbla.lon<=qq) & (eqbla.lat>=p) & (eqbla.lat<=q))
        eqslft_check = eqslfta.where((eqslfta.lon>=pp) & (eqslfta.lon<=qq) & (eqslfta.lat>=p) & (eqslfta.lat<=q))
        equlft_check = equlfta.where((equlfta.lon>=pp) & (equlfta.lon<=qq) & (equlfta.lat>=p) & (equlfta.lat<=q))

        subs_check = subsa.where((subsa.lon>=pp) & (subsa.lon<=qq) & (subsa.lat>=p) & (subsa.lat<=q))
        cape_check = capea.where((capea.lon>=pp) & (capea.lon<=qq) & (capea.lat>=p) & (capea.lat<=q))

        buoy_check = buoya.where((buoya.lon>=pp) & (buoya.lon<=qq) & (buoya.lat>=p) & (buoya.lat<=q))
        preci_check = precia.where((precia.lon>=pp) & (precia.lon<=qq) & (precia.lat>=p) & (precia.lat<=q))


    else:
        continue
    eqblint = np.nanmean(eqbl_check) #takes average of inner box
    eqslftint = np.nanmean(eqslft_check)
    equlftint = np.nanmean(equlft_check)
    subsint = np.nanmean(subs_check)
    capeint = np.nanmean(cape_check)
    buoyint = np.nanmean(buoy_check)
    preciint = np.nanmean(preci_check)

    psler=psl[t] #for use in writing in text
    #looker[17:28] takes the names of each storm
    if factor="EQBL/EQSLFT/EQULFT":
        f=open('Compare_In_Out_Final_EQBL_EQSLFT_EQULFT.txt','a')


        lines = ['{}'.format(year), ' ', '{}'.format(tc_number), ' ', '{}'.format(psler), ' ', '{}'.format(eqblint),' ', '{}'.format(eqblext), ' ', '{}'.format(eqslftint),' ', '{}'.format(eqslftext), ' ', '{}'.format(equlftint),' ', '{}'.format(equlftext), ' ', '{}'.format(looker[17:28])] 
        f.writelines(lines)
        f.write('\n')
        f.close()
    elif factor="CAPE/Subsaturation":
        f=open('Compare_In_Out_Final_Subs_CAPE.txt','a')


        lines = ['{}'.format(year), ' ', '{}'.format(tc_number), ' ', '{}'.format(psler), ' ', '{}'.format(capeint),' ', '{}'.format(capeext), ' ', '{}'.format(subsint),' ', '{}'.format(subsext),' ', '{}'.format(looker[17:28])] 
        f.writelines(lines)
        f.write('\n')
        f.close()
    elif factor="Buoyancy":
        f=open('Compare_In_Out_Final_Buoyancy.txt','a')


        lines = ['{}'.format(year), ' ', '{}'.format(tc_number), ' ', '{}'.format(psler), ' ', '{}'.format(buoyint),' ', '{}'.format(buoyext), ' ', '{}'.format(looker[17:28])] 
        f.writelines(lines)
        f.write('\n')
        f.close()
    elif factor="Precipitation":
        f=open('Compare_In_Out_Final_Precipitation.txt','a')


        lines = ['{}'.format(year), ' ', '{}'.format(tc_number), ' ', '{}'.format(psler), ' ', '{}'.format(preciint),' ', '{}'.format(preciext), ' ', '{}'.format(looker[17:28])] 
        f.writelines(lines)
        f.write('\n')
        f.close()
    l=l+1 #so it chooses the next size
print("Done")

0
1.25
1
1.5
2
1.5
Done


In [379]:
#EST 2am = 6am UTC 6
#EST 8am = 12pm UTC 12
#EST 2pm = 6pm UTC 18
#EST 8pm = 12am UTC 00
#CST 1am = 6am UTC 6
#CST 7am = 12pm UTC 12
#CST 1pm = 6pm UTC 18
#CST 7pm = 12am UTC 00
#MST 12am = 6am UTC 6
#MST 6am = 12pm UTC 12
#MST 12pm = 6pm UTC 18
#MST 6pm = 12am UTC 00
#PST 11pm = 6am UTC 6
#PST 5am = 12pm UTC 12
#PST 11am = 6pm UTC 18
#PST 5pm = 12am UTC 00